In [1]:
import numpy as np, pandas as pd, warnings, os
from tqdm.auto import tqdm

/home/jellybee/anaconda3/envs/llm_311/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Data Load

In [98]:
df = pd.read_csv("../data/서울시대여공구찾기정보.csv", encoding='cp949')

## Preprocessing

In [ ]:
del_col = df.isnull().sum()[df.isnull().sum()/df.shape[0]>0.95].index.tolist()
df = df[[x for x in df.columns if x not in del_col]]
df['생성일시'] = pd.to_datetime(df['생성일시'], format='%Y%m%d%H%M%S')
del df['위치(위도,경도)']
del df['요금']

In [100]:
df['요금(과금기준 + 요금)'].fillna('0원', inplace=True)
def split_period_and_price(x):
    if '1일' in x:
        return pd.Series(['1일', x.replace('1일', '').strip()])
    elif '1시간' in x:
        return pd.Series(['1시간', x.replace('1시간', '').strip()])
    else:
        return pd.Series(['없음', x.strip()])

df[['과금기준', '요금']] = df['요금(과금기준 + 요금)'].apply(split_period_and_price)
del df['요금(과금기준 + 요금)']

/tmp/ipykernel_14207/2815472384.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['요금(과금기준 + 요금)'].fillna('0원', inplace=True)


# Location Info 

In [30]:
import requests

def get_lat_lon(location_name):
    url = 'https://nominatim.openstreetmap.org/search'
    params = {
        'q': location_name,
        'format': 'json'
    }
    headers = {
        'User-Agent': 'Mozilla/5.0 (compatible; ChatGPT-Example/1.0; +http://yourdomain.com)'
    }

    response = requests.get(url, params=params, headers=headers)

    # 응답 확인
    if response.status_code != 200:
        print(f"요청 실패: {response.status_code}")
        print("응답 내용:", response.text)
        return None

    try:
        data = response.json()
        if data:
            lat = data[0]['lat']
            lon = data[0]['lon']
            return float(lat), float(lon)
        else:
            print("위치 정보를 찾을 수 없습니다.")
            return None
    except ValueError as e:
        print("JSON 파싱 오류:", e)
        print("응답 내용:", response.text)
        return None

# 테스트
location = '강남역'
result = get_lat_lon(location)
print(f"{location} 위도/경도: {result}")


강남역 위도/경도: (37.4999348, 127.0269678)
